# Title

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * a
  

## Parameter and environment specification <a id="Parameter_and_environment_specification"></a>

### Command line options <a id="Command_line_options"></a>

SoS allows you to define parameters that accept values from command line options easily. 

#### Optional arguments <a id="Optional_arguments"></a>

Any variable assignment prefixed with `parameter:` can accept values from command  line. The format of such lines are

```
# comment
parameter: var_name = default_value
```

The default value can be integer, float, boolean, string, list of string, or expressions that return values of these types. Other types can be used as long as they can be converted to these types from user-provided values. For example

In [53]:
# path to tool gatk
parameter: gatk_path = '~/bin/GATK'

defines a variable `gatk_path` with default value `'~/bin/GATK'`.

If you execute a workflow without defining `gatk_path`, the default value will be used

In [54]:
%run
parameter: gatk_path = '~/bin/GATK'
print(gatk_path)

~/bin/GATK


You can set this parameter from command line using option `--gatk_path`

In [55]:
%run --gatk_path /my/GATK
parameter: gatk_path = '~/bin/GATK'
print(gatk_path)

/my/GATK


You can also use `-` in place of `_` in command line, although the variable name cannot have `-` in it.

In [56]:
%run --gatk-path /my/GATK
parameter: gatk_path = '~/bin/GATK'
print(gatk_path)

/my/GATK


Actually, because the input is a path, it can be more convenient to create a `path` object

In [57]:
%run --gatk-path /my/GATK
parameter: gatk_path = path('~/bin/GATK')
print(f"gatk_path={gatk_path!r}")

gatk_path=path('/my/GATK')


The default value of the parameter determines the type of values it accepts. For example, the following command yields an error because the parameter expects an integer (from default value `0`) but a string is passed.

In [58]:
%sandbox --expect-error

%run --cutoff zero
parameter: cutoff = 0


---------------------------------------------------------------------------
ArgumentError                             Traceback (most recent call last)
script_7619312906926821593 in <module>
      if "sos_handle_parameter_" in globals():
---->     cutoff = sos_handle_parameter_('cutoff',  0
      )

ArgumentError: argument --cutoff: invalid int value: 'zero'


Simiarly, a list would be created if the parameter has a default value of type list

In [59]:
%run --sample-names A
parameter: sample_names = []
print(sample_names)

['A']


and you can specify default values of type `paths` if you are expecting multiple filenames:

In [60]:
%run --input-files a.txt b.txt
parameter: input_files = paths('a.txt')
print(f"input_files={input_files!r}")

input_files=[path('a.txt'), path('b.txt')]


#### Required arguments <a id="Required_arguments"></a>

In cases where there is no suitable default values and/or command line arguments are mandatory, you can specify a parameter without initial value, and the parameter will be considered required. By default, the parameter will be considered to be of a single value with `str` type.

In [61]:
%sandbox --expect-error

parameter: cutoff 
print(f"cutoff = {cutoff} of type {type(cutoff).__name__}")


---------------------------------------------------------------------------
ArgumentError                             Traceback (most recent call last)
script_5036437366813650181 in <module>
      if "sos_handle_parameter_" in globals():
---->     cutoff = sos_handle_parameter_('cutoff', str)
      print(f"cutoff = {cutoff} of type {type(cutoff).__name__}")

ArgumentError: Argument cutoff of type str is required


In [62]:
%run --cutoff 100
parameter: cutoff 
print(f"cutoff = {cutoff} of type {type(cutoff).__name__}")

cutoff = 100 of type str


If you would like to specify a type, you can use the Python type hint to annotate the variable with a type

In [63]:
%run --cutoff 100
parameter: cutoff : int
print(f"cutoff = {cutoff} of type {type(cutoff).__name__}")

cutoff = 100 of type int


Currently SoS only accepts simple type hints such as `int`, `str`, `float`, `list` (only list of strings are supported), and target types such as `path`, `paths`, `file_target`, and `sos_targets`. For example

In [64]:
%run --bam-files A.bam B.bam
# input bam files
parameter: bam_files : paths
print(bam_files)

A.bam B.bam


It is worth noting that you can specify type hint also for parameters with initial values but SoS will use type of the default values and ignore type hints in this case. For example, in the following case when a parameter with default value 10 are given a type hint of `list`, it will accept input as integers.

In [65]:
%run --cutoff 100
parameter: cutoff : list = 10
print(f"cutoff = {cutoff} of type {type(cutoff).__name__}")

cutoff = 100 of type int


## Further reading

* 